# __Experiments__

In [337]:
import math
import pandas as pd
import numpy as np
import nltk
import time
from collections import Counter, defaultdict
import scipy.stats
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.grid_search import GridSearchCV

In [2]:
data_path = '../data/clean_data_full.csv'
df = pd.read_csv(data_path)
# df.info()

In [3]:
## Num rows
print "num observations:\t", len(df)

num observations:	52778


# Feature Engineering

In [4]:
def unigram_phi(review):
    return Counter(review.split())
def bigram_phi(review):
    return Counter(nltk.bigrams(review.split()))
def unigram_bigram_phi(review):
    return unigram_phi(review) + bigram_phi(review)
def trigram_phi(review):
    return Counter(nltk.trigrams(review.split()))

# Learners

In [5]:
class Baseline:
    def __init__(self):
        self.avg_respsponse = 0
    def fit(self, X, y):
        self.avg_response = np.mean(y)
    def predict(self, X):
        rows, _ = X.shape
        return np.repeat(self.avg_response, rows)

def fit_baseline(X, y):
    """
    Naive baseline return mean training prediction
    """
    mod = Baseline()
    mod.fit(X, y)
    return mod

In [6]:
def fit_linear_regression(X, y):    
    """
    Linear Regression
    """
    mod = LinearRegression(fit_intercept=True, n_jobs = -1)
    mod.fit(X, y)
    return mod

In [7]:
def fit_lasso(X, y, alpha = 0.1, max_iter = 1000):
    lasso = Lasso(alpha = alpha, max_iter = max_iter)
    mod = lasso.fit(X, y)
    return mod

In [8]:
def fit_ridge(X, y, alpha = 0.1, max_iter = 1000):
    lasso = Lasso(alpha = alpha, max_iter = max_iter)
    mod = lasso.fit(X, y)
    return mod

In [73]:
def fit_gbm_regression(X, y,
                       n_estimators = 100,
                       learning_rate = 0.1,
                       max_depth = 1,
                       random_state = 0,
                       loss = "ls"):
    """
    Gradient Boosting Method Regression
    """
    gbm = GradientBoostingRegressor(n_estimators = n_estimators,
                                    learning_rate = learning_rate,
                                    max_depth = max_depth,
                                    random_state = random_state,
                                    loss = "ls")
    mod = gbm.fit(X.toarray(), y)
    return mod

In [141]:
def fit_rf_regression(X, y,
                      n_estimators = 100,
                      max_depth = math.sqrt,
                      max_features = 'auto',
                      n_jobs = -1):
    """
    Gradient Boosting Method Regression
    """
    ## Restrict depth with fn if None is not passed
    ## expect somthing like sqrt() to be passed here
    max_depth = max_depth(X.shape[1]) if max_depth != None else max_depth
    rf = RandomForestRegressor(n_estimators = n_estimators,
                               max_depth = max_depth,
                               max_features = max_features,
                               n_jobs = n_jobs)
    mod = rf.fit(X.toarray(), y)
    return mod

In [11]:
def fit_adaBoost_regression(X, y,
                            n_estimators = 50,
                            learning_rate = 1,
                            loss = "linear"):
    """
    Gradient Boosting Method Regression
    """
    adaBoost = AdaBoostRegressor(n_estimators = n_estimators,
                           learning_rate = learning_rate,
                           loss = loss)
    mod = adaBoost.fit(X.toarray(), y)
    return mod

In [12]:
def build_data_set(data, vectorizer = None, aspect_str = "OVERALL"):
    """
    Aspect ratings cols:
        (7)  review_palate_score      -- score_normalizer = 1
        (8)  review_taste_score       -- score_normalizer = 2
        (11) review_aroma_score       -- score_normalizer = 2
        (14) review_avg_score         -- score_normalizer = 1
        (18) review_overall_score     -- score_normalizer = 4
        (20) review_appearance_score  -- score_normalizer = 1
        
    predict_col :: column for aspect we're predicting, current default is column 18 (OVERALL)
    
    """
    
    ## RateBeer scrape data locations
    ## ------------------------------
    REVIEW_BLOB = 24
    ASPECTS = {
        "PALATE"     : [7, 1],
        "TASTE"      : [8, 2],
        "AROMA"      : [11, 2],
        "AVERAGE"    : [14, 1],
        "OVERALL"    : [18, 4],
        "APPEARANCE" : [20, 1]
    }
    assert aspect_str in ASPECTS
        
    aspect_column = ASPECTS[aspect_str][0]      ## Get aspect rating column
    aspect_normalizer = ASPECTS[aspect_str][1]  ## Get aspect normalizer
    labels = []                                 ## Ratings
    feat_dicts = []                             ## Features
    raw_examples = []                           ## Review strings
    data_values = data.values                   ## Data from pandas df
    for row in data.values:
        review, score = row[REVIEW_BLOB], row[aspect_column]
        score = float(score) / aspect_normalizer

        ## Safety check
        if not isinstance(review, basestring):
            print 'weird review:\t', review
            
#         feat_dicts.append(phi(review))
        labels.append(score)
        raw_examples.append(review)
        
    # In training, we want a new vectorizer:
    if vectorizer == None:
        vectorizer = DictVectorizer(sparse=True)
        feat_matrix = vectorizer.fit_transform(feat_dicts)
    # In assessment, we featurize using the existing vectorizer:
    else:
        feat_matrix = vectorizer.fit_transform(raw_examples)

    return {'X'            : feat_matrix, 
            'y'            : labels, 
            'vectorizer'   : vectorizer, 
            'raw_examples' : raw_examples,
            'feature_names' : vectorizer.get_feature_names()}

In [180]:
def experiment(data,
               model = fit_baseline,
               phi = None,
               assess_data = None,
               train_size = 0.9,
               score_metric = mean_squared_error,
               verbose = False):
    ## Check that we're running a model
    assert(model != None)
    
    ## Timing
    start_time = time.time()
    
    ## Build data set
    X_train = data['X'] 
    y_train = data['y']
    vectorizer = data['vectorizer']
    feature_names = data['feature_names']

    ## Test-train split
    if assess_data == None:
        X_train, X_assess, y_train, y_assess = train_test_split(
                X_train, y_train, train_size = train_size)
    ## Only use for test-set
    else:
        assess = build_data_set(assess_data, phi, vectorizer = vectorizer)
        X_assess, y_assess = assess['X'], assess['y']
        

    ## Model data
    mod = model(X_train, y_train)
    predictions = mod.predict(X_assess.toarray())
    
    ## Timing
    run_time = time.time() - start_time

    if verbose:
        print "\tExperiment information"
        print '\t======================='
        print "\ttrained:\t", model.__name__
        print "\tnum training observations:\t", X_train.shape[0]
        print "\tnum training features:\t", X_train.shape[1]
        print "\tscore_metric:\t", score_metric.__name__
        print "\trun time: ", run_time
#         if model.__name__ == 'fit_linear_regression' or\
#             model.__name__ == 'fit_lasso' or\
#             model.__name__ == 'fit_ridge':
# #             print feature_names[1:10]
# #             print mod.coef_[1]
        print "y_assess[1:10]:\t", y_assess[1:10]
        print "predictions[1:10]:\t", predictions[1:10]
    
    ## Return results dict - will write to csv
    res = {
        "score"        : score_metric(y_assess, predictions),
        "score_name"   : score_metric.__name__,
        "predictions"  : predictions,
        "y_assess"     : y_assess
    }
    return res
    

In [192]:
def data_reader(train_path = data_path, n = 1000):
    df = pd.read_csv(data_path)
    return df.sample(n)

# def dev_reader(test_path, n = 100):
#     df = pd.read_csv(data_path)
#     return df.sample(n)

# Vectorizer's encode phi...

In [27]:
## bigram_vectorizer
## -----------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
bigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(2, 2), min_df = 3, max_features = 5000)

In [28]:
## unigram_vectorizer
## -----------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
unigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(1, 1), min_df = 3, max_features = 5000)

In [29]:
## trigram_vectorizer
## ------------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
trigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(3, 3), min_df = 3, max_features = 5000)

In [303]:
def set_vectorizer(ngram_range = (1,1),
                   min_df = 3,
                   sample_size = 10000,
                   stop_words = 'english',
                   features_prop = 0.05,
                   max_features = 2000,
                   lowercase = True,
                   binary = False):

    max_features = int(sample_size * features_prop) if max_features == None else max_features
    
    return CountVectorizer(analyzer='word',
                           ngram_range = ngram_range,
                           min_df = min_df,
                           max_features = max_features,
                           stop_words = stop_words,
                           binary = binary,
                           lowercase = lowercase)

# Data

In [242]:
# def build_data_set(data, phi, vectorizer = None, aspect_str = "OVERALL"):

n_samples = 40000
vectorizer = set_vectorizer(n_gram = "unigram", sample_size = n_samples, features_prop = 0.05, max_features = None)
train_d = data_reader(data_path, n = n_samples)
built_data_set = build_data_set(train_d, vectorizer = vectorizer)

# Prelim runs

### Baseline

In [196]:
baseline_results = experiment(data = built_data_set,
           model = fit_baseline,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print baseline_results['score_name']
print baseline_results['score']

	Experiment information
	trained:	fit_baseline
	num training observations:	9000
	num training features:	500
	score_metric:	mean_squared_error
	run time:  0.00888109207153
y_assess[1:10]:	[5.0, 3.25, 2.5, 3.5, 4.0, 3.5, 3.0, 3.0, 3.75]
predictions[1:10]:	[ 3.45480556  3.45480556  3.45480556  3.45480556  3.45480556  3.45480556
  3.45480556  3.45480556  3.45480556]
-----------------------------
mean_squared_error
0.887323857253


### Linear regresssion

In [246]:
## Note: running time ~ 1 minute @ 40k observations
## Note on number of features :: lookes like LM with n = 40,000 obs prefers 1,200 features
lm_results = experiment(data = built_data_set,
           model = fit_linear_regression,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print lm_results['score_name']
print lm_results['score']

	Experiment information
	trained:	fit_linear_regression
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.69296002388
y_assess[1:10]:	[4.0, 5.0, 4.5, 3.75, 3.25, 4.25, 4.75, 4.5, 4.5]
predictions[1:10]:	[ 4.10252529  3.39662777  3.59958548  3.54214488  3.47564997  3.27873549
  3.77199107  3.58033956  3.94887898]
-----------------------------
mean_squared_error
0.678707950421


### Regularized Regression

In [119]:
# n_samples = 20000
# vectorizer = set_vectorizer(n_gram = "unigram", sample_size = n_samples, features_prop = 1, max_features = None)
# train_d = train_reader(data_path, n = n_samples)
# built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = 'TASTE')

#### Ridge regression

In [202]:
## Note: running time few seconds @ 40k observations
ridge_results = experiment(data = built_data_set,
           model = fit_ridge,
           verbose = True, 
           score_metric = mean_squared_error)
print "-----------------------------"
print ridge_results['score_name']
print ridge_results['score']

	Experiment information
	trained:	fit_ridge
	num training observations:	9000
	num training features:	6374
	score_metric:	mean_squared_error
	run time:  0.178368806839
y_assess[1:10]:	[4.0, 4.25, 4.0, 3.5, 4.25, 4.25, 2.5, 2.5, 4.0]
predictions[1:10]:	[ 3.47438889  3.47438889  3.47438889  3.47438889  3.47438889  3.47438889
  3.47438889  3.47438889  3.47438889]
-----------------------------
mean_squared_error
0.940097873457


#### Lasso

In [203]:
## Note: running time few seconds @ 40k observations
lasso_results = experiment(data = built_data_set,
           model = fit_lasso,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print lasso_results['score_name']
print lasso_results['score']

	Experiment information
	trained:	fit_lasso
	num training observations:	9000
	num training features:	6374
	score_metric:	mean_squared_error
	run time:  0.138798952103
y_assess[1:10]:	[4.5, 3.75, 4.25, 3.0, 3.75, 0.25, 4.0, 1.25, 3.25]
predictions[1:10]:	[ 3.47011111  3.47011111  3.47011111  3.47011111  3.47011111  3.47011111
  3.47011111  3.47011111  3.47011111]
-----------------------------
mean_squared_error
1.02048034568


### Random Forest

In [250]:
## Note: run time ~3min with 10K obs and 10 trees
##                ~8min with 40K obs and 0.05 * 40K (2000)features and 10 trees
##                ~
rf_results = experiment(data = built_data_set,
           model = fit_rf_regression,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print rf_results['score_name']
print rf_results['score']

### GBM regression

In [251]:
# def fit_gbm_regression(X, y,
#                        n_estimators = 10,
#                        learning_rate = 0.1,
#                        max_depth = 1,
#                        random_state = 0,
#                        loss = "ls"):

# Note: running time is currently ver long
experiment(data = built_data_set,
           model = fit_gbm_regression,
           verbose = True,
           score_metric = mean_squared_error)

### Ada Boost regression

In [ ]:
# Note: running time is currently ver long
experiment(data = built_data_set,
           model = fit_adaBoost_regression,
           verbose = True,
           score_metric = mean_squared_error)

## Parameter tuning

In [213]:
def tune_learner_params(X, y, basemod, cv, param_grid, scoring = None): 
    """
    Description
        
    """    
    # Find the best model within param_grid:
    crossvalidator = GridSearchCV(basemod, param_grid, cv = cv, scoring=scoring, n_jobs = -1)
    crossvalidator.fit(X, y)
    # Report some information:
    print("All results", crossvalidator.grid_scores_)
    print("Best params", crossvalidator.best_params_)
    print("Best score: %0.03f" % crossvalidator.best_score_)

    return crossvalidator.best_estimator_

In [223]:
def fit_lasso_cv(X, y):
    """
    
    """    
    basemod = Lasso()
    cv = 2
    param_grid = {
        'alpha': [0.01, 0.05, 0.1, 0.2, 0.5], 
        'max_iter': [2, 10, 25, 50, 500]
                 }    
    return tune_learner_params(X, y, basemod, cv, param_grid)

In [224]:
lasso_cv_res = experiment(data = built_data_set,
                          model = fit_lasso_cv,
                          verbose = True,
                          score_metric = mean_squared_error)

('All results', [mean: 0.12776, std: 0.00227, params: {'alpha': 0.01, 'max_iter': 2}, mean: 0.12659, std: 0.00218, params: {'alpha': 0.01, 'max_iter': 10}, mean: 0.12659, std: 0.00218, params: {'alpha': 0.01, 'max_iter': 25}, mean: 0.12659, std: 0.00218, params: {'alpha': 0.01, 'max_iter': 50}, mean: 0.12659, std: 0.00218, params: {'alpha': 0.01, 'max_iter': 500}, mean: 0.01084, std: 0.00108, params: {'alpha': 0.05, 'max_iter': 2}, mean: 0.01079, std: 0.00111, params: {'alpha': 0.05, 'max_iter': 10}, mean: 0.01079, std: 0.00111, params: {'alpha': 0.05, 'max_iter': 25}, mean: 0.01079, std: 0.00111, params: {'alpha': 0.05, 'max_iter': 50}, mean: 0.01079, std: 0.00111, params: {'alpha': 0.05, 'max_iter': 500}, mean: -0.00000, std: 0.00000, params: {'alpha': 0.1, 'max_iter': 2}, mean: -0.00000, std: 0.00000, params: {'alpha': 0.1, 'max_iter': 10}, mean: -0.00000, std: 0.00000, params: {'alpha': 0.1, 'max_iter': 25}, mean: -0.00000, std: 0.00000, params: {'alpha': 0.1, 'max_iter': 50}, mean

In [226]:
print lasso_cv_res['score']

0.915768392253


In [244]:
def fit_rf_cv(X, y):
    """
    
    """    
    basemod = RandomForestRegressor()
    cv = 2
    param_grid = {
        'n_estimators': [100, 500], 
        'max_features': ['log2']
                 }    
    return tune_learner_params(X, y, basemod, cv, param_grid)

In [323]:
## Note  :: got MSE of 0.68 with RF 
rf_cv_res = experiment(data = built_data_set,
                          model = fit_rf_cv,
                          verbose = True,
                          score_metric = mean_squared_error)

In [ ]:
rf_cv_res['score']

## Tune num features

In [36]:
def tune_max_features(n_observations = 10000,
                      n_gram = "unigram",
                      tuning_values = [0.05, 0.10, 0.2, 0.3],
                      model = fit_rf_regression,
                      score_metric = mean_squared_error,
                      verbose = False):

    start_time = time.time()
    
    res = defaultdict(float)
    for tuning_value in tuning_values:
        if verbose:
            print "Currently tuning:\t", model.__name__
            print "n_gram:\t", n_gram
            print "tuning_value:\t", tuning_value
            
        vectorizer = set_vectorizer(n_gram = n_gram,
                                sample_size = n_observations,
                                features_prop = tuning_value)
        train_d = train_reader(data_path, n = n_samples)
        built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = 'TASTE')
        res[str(tuning_value)] = experiment(data = built_data_set,
                                            model = model,
                                            verbose = False,
                                            score_metrics = [mean_squared_error])

    run_time = time.time() - start_time    
    print "runtime for " + str(n_observations) + ":\t", run_time
    return res

In [247]:
## Records
# n = 5000; runtime = 235sec; best 0.3 (0.69)
# n = 2000; runtime = 60sec
# n = 10000, alpha = 0.2, mse = 0.63

# tune_max_features(n_observations = 10000, n_gram = "unigram", verbose = True)

## General cross-validated score

In [355]:
## Data
n_samples = 40000
vectorizer = set_vectorizer(ngram_range = (1, 1),
                            sample_size = n_samples,
                            features_prop = 0.05,
                            max_features = None,
                            lowercase = True,
                            binary = True)
train_d = data_reader(data_path, n = n_samples)
built_data_set = build_data_set(train_d, vectorizer = vectorizer)

In [368]:
def fit_model_cv(X, y, model, cv = 10, scoring = 'mean_squared_error'):
    """
    
    """    
    scores = cross_val_score(model,
                             X = X,
                             y = y,
                             scoring = scoring,
                             cv = cv,
                             n_jobs = -1)
    return scores * -1 ## cross_val_score flips the sign


### GBM model

In [358]:
gbm_params = {
    'n_estimators': 500,
    'max_depth': 4,
    'min_samples_split': 1,
    'learning_rate': 0.01,
    'loss': 'ls'
}
gbm_model = GradientBoostingRegressor(**gbm_params)

In [359]:
rf_params = {
    'n_estimators' : 500,
    'max_features' : 'log2',
    'n_jobs'       : -1
}
rf_model = RandomForestRegressor(**rf_params)

In [360]:
lm_model = LinearRegression()

In [361]:
lm_cv_res = fit_model_cv(built_data_set['X'], built_data_set['y'], model = lm_model, cv = 10)
print 'mean', np.mean(lm_cv_res)
print 'var', np.var(lm_cv_res)

mean 0.661254254169
var 0.000218697071569


In [367]:
start_time = time.time()
rf_cv_res = fit_model_cv(built_data_set['X'], built_data_set['y'], model = rf_model, cv = 2)
run_time = time.time() - start_time
print 'mean:\t', np.mean(rf_cv_res)
print 'var:\t', np.var(rf_cv_res)
print 'runtime:\t', run_time

mean:	0.665621206226
var:	5.84956498855e-06
runtime:	186.072751999


In [366]:
start_time = time.time()
gbm_cv_res = fit_model_cv(built_data_set['X'].toarray(), built_data_set['y'], model = gbm_model, cv = 2)
run_time = time.time() - start_time
print 'mean:\t', np.mean(gbm_cv_res)
print 'var:\t', np.var(gbm_cv_res)
print 'runtime:\t', run_time

KeyboardInterrupt: 

# Between aspects

In [248]:
ngram = 'unigram'
ASPECTS = ['OVERALL', 'TASTE', 'AROMA', 'PALATE', 'APPEARANCE']
for aspect in ASPECTS:    
    n_samples = 40000
    vectorizer = set_vectorizer(n_gram = ngram, sample_size = n_samples, features_prop = 0.05, max_features = None)
    train_d = data_reader(data_path, n = n_samples)
    built_data_set = build_data_set(train_d, vectorizer = vectorizer)
    print "\n-----------------"
    print "ngram:\t", ngram
    print "curr aspect:\t", aspect
    
    # Current model
    results = experiment(data = built_data_set,
                         model = fit_linear_regression,
                         verbose = True,
                         score_metric = mean_squared_error)
    print "mse:\t", results['score']
    


-----------------
ngram:	unigram
curr aspect:	OVERALL
	Experiment information
	trained:	fit_linear_regression
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.698040962219
y_assess[1:10]:	[4.5, 4.25, 4.0, 0.25, 3.75, 4.75, 3.25, 3.25, 3.0]
predictions[1:10]:	[ 3.94613552  4.53600669  4.00284222  3.46959722  3.10406295  3.84065894
  3.24267995  3.72572006  3.62688588]
mse:	0.635508621105

-----------------
ngram:	unigram
curr aspect:	TASTE
	Experiment information
	trained:	fit_linear_regression
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.692892074585
y_assess[1:10]:	[4.25, 3.0, 3.25, 1.25, 3.5, 4.25, 5.0, 3.0, 1.75]
predictions[1:10]:	[ 4.10865449  3.33297045  3.09999102  1.28320703  3.3647986   4.60556742
  3.95957174  3.75462826  3.42502059]
mse:	0.713278923193

-----------------
ngram:	unigram
curr aspect:	AROMA
	Experiment information
	trained:	fit_linear

# Multiple runs

In [128]:
learners = [fit_linear_regression, fit_lasso, fit_gbm_regression]
vectorizers = [unigram_vectorizer, bigram_vectorizer]
vectorizer_names = ["unigram", "bigram"]

In [132]:
for learner in learners:
    print "\n============================="
    print "Fitting:\t", learner.__name__
    for vectorizer, name in zip(vectorizers, vectorizer_names):
        print "-------"
        print "vectorizer:\t", name
        mse = experiment(train_d,
                         learner,
                         unigram_phi,
                         vectorizer = vectorizer,
                         verbose = True)
        print "mse:\t", mse


Fitting:	fit_linear_regression
-------
vectorizer:	unigram


KeyboardInterrupt: 

# Model testing

In [283]:
def run_model(d_sample, model, phi, n_samples = 100):
    res = [experiment(data, model, phi) for _ in range(n_samples)]
    return res

In [284]:
glm_res = run_model(train, fit_linear_regression, trigram_phi)
gbm_res = run_model(train, fit_gbm_regression, trigram_phi)
lasso_res = run_model(train, fit_lasso, trigram_phi)

In [285]:
print "glm:", np.mean(glm_res), np.var(glm_res)
print "gbm:", np.mean(gbm_res), np.var(gbm_res)
print "lasso", np.mean(lasso_res), np.var(lasso_res)

glm: 5.09104430619 1.65291139452
gbm: 5.4193393923 2.1477179012
lasso 5.5710284521 1.65857267327


In [288]:
print scipy.stats.wilcoxon(glm_res, gbm_res)[1]
print scipy.stats.wilcoxon(glm_res, lasso_res)[1]
print scipy.stats.wilcoxon(gbm_res, lasso_res)[1]

0.0933661297989
0.0212407456292
0.5025560931
